In [2]:
%matplotlib inline

In [1]:
import numpy as np
from mtpy.utils import interp, nc, lib

No handlers could be found for logger "mtpy.utils.decorator"


GDAL_DATA environment variable not set. Please see https://trac.osgeo.org/gdal/wiki/FAQInstallationAndBuilding#HowtosetGDAL_DATAvariable for more information.
2018-12-18T11:23:36 - mtpy.utils.decorator - WARNING - GDAL_DATA environment variable is not set
GDAL_DATA environment variable not set. Please see https://trac.osgeo.org/gdal/wiki/FAQInstallationAndBuilding#HowtosetGDAL_DATAvariable for more information.
2018-12-18T11:23:36 - mtpy.utils.decorator - WARNING - GDAL_DATA environment variable is not set
GDAL_DATA environment variable not set. Please see https://trac.osgeo.org/gdal/wiki/FAQInstallationAndBuilding#HowtosetGDAL_DATAvariable for more information.


In [5]:
# the input data "winGlink_3dmod_input.xyzv" had no information about the crs.
# so, we had to hard code it.
# epsg 32753 represents the data area are in WGS84/ UTM 53S

source_epsg = 32753
wgs84_epsg = 4326

# make a grid that covers the whole area of all the points
# with desired resolution. The grid epsg we want is 4326 which is global and in lat/lon.
geo_grid = lib.Grid3D.from_extent_and_resolution(left=(131.5, -31., -4500.),
                                                 right=(132.5, -30., -125.),
                                                 resolution=(0.005, 0.005, 100.),
                                                 epsg=wgs84_epsg)
# utm points are poivins flattened in our data crs for IDW interpolation
utm_points = geo_grid.flatten().to_epsg(source_epsg)

In [6]:
utm_points.points.shape

(1760000, 3)

In [7]:
# the input data are in Col1:lat(utm),Col2:lon(utm)
# Col3:depth(m),Col4:resistivity.

arr = np.loadtxt('winGlink_3dmod_input.xyzv')

# input data hes been fixed for false origin
# Points3D object is a collection of 3D points that remembers its epsg

false_easting = 500000.0
false_northing = 10000000.0

source_points = lib.Points3D(np.array([arr[:, 0] - false_easting, arr[:, 1]- false_northing, arr[:, 2]]).T, source_epsg)


In [8]:
source_val = arr[:, 3]
#print(source_val.shape)

source_in_geo = source_points.to_epsg(wgs84_epsg)

# The upper layer has very high resistivity to distinguish
# it from the other layer, so masking has been used
mask = source_in_geo.clipping_mask(geo_grid)
source_points.points = source_points.points[mask]

source_val = source_val[mask]

In [9]:
#resistivity = interp.nearest(source_points, source_val, utm_points)
resistivity = interp.IDW(source_points, source_val, utm_points)

In [10]:
resistivity = resistivity.reshape(geo_grid.shape[:3])

print(resistivity.shape)

(200, 200, 44)


In [11]:
# source points are our data points and geo_grid is the
# grid where we want our values.
nc.write_resistivity_grid('IDW.nc', wgs84_epsg, geo_grid.y, geo_grid.x, geo_grid.z, resistivity.transpose([2, 0, 1]))

